參考文章

[[機器學習專案] Kaggle競賽-鐵達尼號生存預測(Top 3%)](https://yulongtsai.medium.com/https-medium-com-yulongtsai-titanic-top3-8e64741cc11f)  
[[資料分析&機器學習] 第4.1講 : Kaggle競賽-鐵達尼號生存預測(前16%排名)](https://medium.com/jameslearningnote/%E8%B3%87%E6%96%99%E5%88%86%E6%9E%90-%E6%A9%9F%E5%99%A8%E5%AD%B8%E7%BF%92-%E7%AC%AC4-1%E8%AC%9B-kaggle%E7%AB%B6%E8%B3%BD-%E9%90%B5%E9%81%94%E5%B0%BC%E8%99%9F%E7%94%9F%E5%AD%98%E9%A0%90%E6%B8%AC-%E5%89%8D16-%E6%8E%92%E5%90%8D-a8842fea7077)  
[Titanic - Exploratory Analysis (with PCA)](https://www.kaggle.com/code/tiagoantao/titanic-exploratory-analysis-with-pca/notebook)  
[Airplane Crash Data Since 1908](https://www.kaggle.com/datasets/cgurkan/airplane-crash-data-since-1908)  
[[Day 15] 機器學習常勝軍 - XGBoost](https://ithelp.ithome.com.tw/articles/10273094?sc=iThomeR)

In [164]:
#系統警告處理工具，了解後再使用
import warnings
warnings.simplefilter('ignore')

In [165]:
#子進程相關，了解後再使用
from subprocess import check_output
#print(check_output(["ls", "../03_topic"]).decode("utf8"))

In [3]:
#常用套件
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#機器學習套件組
from sklearn import tree
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold, learning_curve, train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import f_regression
from sklearn.linear_model import LinearRegression
#統計套件組
import statsmodels.api as sm
from scipy import stats

In [4]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
submit = pd.read_csv("gender_submission.csv")

In [168]:
data = pd.concat([train,test] , ignore_index = True)
data.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1304,1305,NaN,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
1305,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
1306,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
1307,1308,NaN,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S
1308,1309,NaN,3,"Peter, Master. Michael J",male,NaN,1,1,2668,22.3583,NaN,C


In [169]:
#檢查資料的缺失狀況
#顯示整包資料列數欄數
nans = {}
for colname in data.columns:
    nans[colname] = data[data[colname].isnull()].shape[0]
print(nans)
print(data.shape[0],data.shape[1])

#Fare跟Embarked缺失值少量可以直接抓掉，但age需要填補

{'PassengerId': 0, 'Survived': 418, 'Pclass': 0, 'Name': 0, 'Sex': 0, 'Age': 263, 'SibSp': 0, 'Parch': 0, 'Ticket': 0, 'Fare': 1, 'Cabin': 1014, 'Embarked': 2}
1309 12


In [170]:
print(data.corr())

             PassengerId  Survived    Pclass       Age     SibSp     Parch  \
PassengerId     1.000000 -0.005007 -0.038354  0.028814 -0.055224  0.008942   
Survived       -0.005007  1.000000 -0.338481 -0.077221 -0.035322  0.081629   
Pclass         -0.038354 -0.338481  1.000000 -0.408106  0.060832  0.018322   
Age             0.028814 -0.077221 -0.408106  1.000000 -0.243699 -0.150917   
SibSp          -0.055224 -0.035322  0.060832 -0.243699  1.000000  0.373587   
Parch           0.008942  0.081629  0.018322 -0.150917  0.373587  1.000000   
Fare            0.031428  0.257307 -0.558629  0.178740  0.160238  0.221539   

                 Fare  
PassengerId  0.031428  
Survived     0.257307  
Pclass      -0.558629  
Age          0.178740  
SibSp        0.160238  
Parch        0.221539  
Fare         1.000000  


In [171]:
#對data加入欄位 倉位 年齡 家人數
#性別代碼
data['Sex_Code'] = data['Sex'].map({'female' : 1, 'male' : 0}).astype('int')

#艙房處理：位在BCDE甲板的有較高的存活率，故建立Cbcde虛擬變數
data['Cnum'] = data['Cabin'].str.slice(0,1)
data['Cnum'] = data['Cnum'].fillna('ZZ')
data['Cnum'] = data['Cnum'].replace('T','ZZ')
data['Cbcde'] = data['Cnum'].apply(lambda x: 1 if x in ['B','C','D','E'] else 0)

#家人變數
data['Fnum'] = data['SibSp'] + data['Parch']
data['FYN'] = data['Fnum'].apply(lambda x: min(x,1))
data['F234'] = data['Fnum'].apply(lambda x: 1 if x in [2,3,4] else 0)

#年齡變數：對小於16歲取虛擬變數
data['A16'] = data['Age'].apply(lambda x: 1 if x < 16 else 0)

#姓名首字母
data['InName'] = data['Name'].str.slice(0,1)

#艙等虛擬變數處理
data['P1'] = data['Pclass'].apply(lambda x: 1 if x == 1 else 0)
data['P2'] = data['Pclass'].apply(lambda x: 1 if x == 2 else 0)
data['P3'] = data['Pclass'].apply(lambda x: 1 if x == 3 else 0)

In [172]:
print(data.corr())

             PassengerId  Survived    Pclass       Age     SibSp     Parch  \
PassengerId     1.000000 -0.005007 -0.038354  0.028814 -0.055224  0.008942   
Survived       -0.005007  1.000000 -0.338481 -0.077221 -0.035322  0.081629   
Pclass         -0.038354 -0.338481  1.000000 -0.408106  0.060832  0.018322   
Age             0.028814 -0.077221 -0.408106  1.000000 -0.243699 -0.150917   
SibSp          -0.055224 -0.035322  0.060832 -0.243699  1.000000  0.373587   
Parch           0.008942  0.081629  0.018322 -0.150917  0.373587  1.000000   
Fare            0.031428  0.257307 -0.558629  0.178740  0.160238  0.221539   
Sex_Code       -0.013406  0.543351 -0.124617 -0.063645  0.109609  0.213125   
Cbcde           0.009496  0.312723 -0.706410  0.300433  0.006164  0.034543   
Fnum           -0.031437  0.016639  0.050027 -0.240229  0.861952  0.792296   
FYN            -0.028546  0.203367 -0.147393 -0.128755  0.591077  0.549022   
F234            0.032949  0.168088 -0.034037 -0.167703  0.253748

In [151]:
#對Age做處理，準備建模資料
full_age = data[data['Age'].notnull()]
print(full_age.shape)
print(full_age.dropna(subset=['Fare', 'Embarked'],how='any').shape)
full_age = full_age.dropna(subset=['Fare', 'Embarked'],how='any')
#print(full_age.head())
print(full_age['Age'].describe())

(1046, 23)
(1043, 23)
count    1043.000000
mean       29.813202
std        14.366254
min         0.170000
25%        21.000000
50%        28.000000
75%        39.000000
max        80.000000
Name: Age, dtype: float64


In [149]:
#考慮使用其他因子對年齡做預測
#解釋力不足，不與使用
#程式碼留存
x = full_age[['Sex_Code','Pclass','Fnum']]
y = full_age['Age']
#x = sm.add_constant(x)

X2 = sm.add_constant(x)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

'''
model = LinearRegression()
model.fit(x,y)
predict = model.predict(x)
#plt.plot(x,predict,c="red")
#plt.scatter(x,y)
#plt.show()
#print(x,y)

mse = np.mean((model.predict(x) - y) ** 2)
r_squared = model.score(x, y)
adj_r_squared = r_squared - (1 - r_squared) * (x.shape[1] / (x.shape[0] - x.shape[1] - 1))

# 印出模型績效
print(mse)
print(r_squared)
print(adj_r_squared)
print(f_regression(x, y)[1])'''

                            OLS Regression Results                            
Dep. Variable:                    Age   R-squared:                       0.224
Model:                            OLS   Adj. R-squared:                  0.222
Method:                 Least Squares   F-statistic:                     99.90
Date:                Wed, 21 Dec 2022   Prob (F-statistic):           8.09e-57
Time:                        18:29:32   Log-Likelihood:                -4126.8
No. Observations:                1043   AIC:                             8262.
Df Residuals:                    1039   BIC:                             8281.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         48.2359      1.199     40.236      0.0

/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


'\nmodel = LinearRegression()\nmodel.fit(x,y)\npredict = model.predict(x)\n#plt.plot(x,predict,c="red")\n#plt.scatter(x,y)\n#plt.show()\n#print(x,y)\n\nmse = np.mean((model.predict(x) - y) ** 2)\nr_squared = model.score(x, y)\nadj_r_squared = r_squared - (1 - r_squared) * (x.shape[1] / (x.shape[0] - x.shape[1] - 1))\n\n# 印出模型績效\nprint(mse)\nprint(r_squared)\nprint(adj_r_squared)\nprint(f_regression(x, y)[1])'

In [179]:
#print(data['Age'].describe())
#print(data['Fare'].describe())
#print(data['Embarked'].describe())
print(data['Age'].median(),data['Fare'].median())

30.0 14.4542


In [180]:
#直接對Age填入中位數，測試使用fillna
data['Age'].fillna(value=30, inplace=True)
data['Fare'].fillna(value=14.45, inplace=True)
data['Embarked'].fillna(value='S', inplace=True)
nans = {}
for colname in data.columns:
    nans[colname] = data[data[colname].isnull()].shape[0]
print(nans)

{'PassengerId': 0, 'Survived': 418, 'Pclass': 0, 'Name': 0, 'Sex': 0, 'Age': 0, 'SibSp': 0, 'Parch': 0, 'Ticket': 0, 'Fare': 0, 'Cabin': 1014, 'Embarked': 0, 'Sex_Code': 0, 'Cnum': 0, 'Cbcde': 0, 'Fnum': 0, 'FYN': 0, 'F234': 0, 'A16': 0, 'InName': 0, 'P1': 0, 'P2': 0, 'P3': 0}


In [181]:
# split training set the testing set
train = data[:len(train)]
test = data[len(train):]
test.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,Cnum,Cbcde,Fnum,FYN,F234,A16,InName,P1,P2,P3
891,892,NaN,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,...,ZZ,0,0,0,0,0,K,0,0,1
892,893,NaN,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,...,ZZ,0,1,1,0,0,W,0,0,1
893,894,NaN,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,...,ZZ,0,0,0,0,0,M,0,1,0


In [182]:
train['Survived'].sum()

342.0

In [41]:
#for i in ['Sex_Code','Pclass','Cbcde','Fnum','A16','Embarked','Cnum']:
#    pvt1 = train.pivot_table(values='PassengerId',index=i, columns='Survived',
#                            aggfunc={'PassengerId':np.count_nonzero})
#    pvt2 = train.pivot_table(values='PassengerId',index=i, #columns='Survived',
#                            aggfunc={'PassengerId':np.count_nonzero})
#    print(pvt1,pvt2)

In [6]:
#製作簡易的因子影響存活率分析表格，程式碼留存
#改factor就可以觀察不同因子
factor = 'Pclass'

sex_group = train[[factor, 'Survived']].groupby(factor)
sex_agg = sex_group.agg(['sum', 'count'])['Survived']
sex_agg['rate'] = sex_agg.apply(lambda row: row['sum'] / row['count'], axis=1)
sex_agg

,sum,count,rate
Pclass,,,
1,136,216,0.629630
2,87,184,0.472826
3,119,491,0.242363


In [157]:
### 以下開始進行建模
X = train.drop(labels=['Survived','PassengerId'],axis=1)
Y = train['Survived']

In [158]:
# Show Baseline
Base = ['Sex_Code','Pclass','Cbcde','Fnum','A16']
Base_Model = RandomForestClassifier(random_state=2,n_estimators=250,min_samples_split=20,oob_score=True)
Base_Model.fit(X[Base], Y)
print('Base oob score :%.5f' %(Base_Model.oob_score_))

Base oob score :0.82267


In [159]:
# 第三次提交：達到29%
# submission if you want
# submits
X_Submit = test.drop(labels=['PassengerId'],axis=1)

Base_pred = Base_Model.predict(X_Submit[Base])

submit = pd.DataFrame({"PassengerId": test['PassengerId'],
                      "Survived":Base_pred.astype(int)})
submit.to_csv("submit_forest4.csv",index=False)

In [91]:
# 嘗試對決策樹視覺化，失敗，程式碼留存
# 提取一個決策樹
estimator = Base_Model.estimators_[0]
from sklearn.tree import export_graphviz
# 導出為dot 文件
export_graphviz(estimator, out_file='tree.dot', 
                feature_names = Base, class_names = "Survived", 
                rounded = True, proportion = False, precision = 2, filled = True) 
# 用系統命令轉為PNG文件(需要 Graphviz) 
from subprocess import call
call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600']) 
# 在jupyter notebook中展示 
from IPython.display import Image
Image(filename = 'tree.png')

FileNotFoundError: [Errno 2] No such file or directory: 'dot'

In [24]:
#讀取已經製作好的生存對照表基底
surv_base = pd.read_csv("survival_table_base.csv")

In [25]:
#產出生存對照表
surv_table = surv_base
surv_table["Survived"] = Base_Model.predict(surv_table[Base])
surv_table.to_csv("surv_table_v1.csv",index=False)

1026.0


In [26]:
#檢視生存對照表生存狀況
print(surv_table["Survived"].sum(),surv_table.shape[0],surv_table.shape[1])

1026.0 1260 17


In [219]:
#嘗試其他建模方法：XGBOOST
xg_base = ['Age', 'Fare', 'Sex_Code', 'Pclass', 'Cbcde']

train_X = train[xg_base]
train_Y = train.Survived

#!pip install xgboost
from xgboost import XGBClassifier

# 建立 XGBClassifier 模型
xgboostModel = XGBClassifier(n_estimators=100, learning_rate= 0.3)
# 使用訓練資料訓練模型
xgboostModel.fit(train_X, train_Y)
# 使用訓練資料預測分類
predicted = xgboostModel.predict(train_X)

# 預測成功的比例
print('訓練集: ',xgboostModel.score(train_X,train_Y))

訓練集:  0.9551066217732884


In [220]:
# 第四次提交：XGBOOST
# 成績稍微低於隨機森林
# submission if you want
# submits
X_Submit = test.drop(labels=['PassengerId'],axis=1)

Base_pred = xgboostModel.predict(X_Submit[xg_base])

submit = pd.DataFrame({"PassengerId": test['PassengerId'],
                      "Survived":Base_pred.astype(int)})
submit.to_csv("submit_xg1.csv",index=False)